In [10]:
# Import necessary libraries
import os
import pandas as pd
import warnings

# Ignore specific warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")


In [11]:
# Define the path to your data folder
data_folder = 'data'  # Make sure to adjust the path to where your CSVs are stored
csv_files = [os.path.join(data_folder, file) for file in os.listdir(data_folder) if file.endswith('.csv')]

# Load and concatenate all CSV files into one DataFrame
data_frames = [pd.read_csv(file) for file in csv_files if os.path.getsize(file) > 0]
hotel_data = pd.concat(data_frames, ignore_index=True)

# Display the first few rows to verify the data has been loaded correctly
print(hotel_data.head())


  ﻿IDS_DATE  Day  Rooms  OOO  StayOver  Arrivals  DueOut  Available  \
0   10/1/22  Sat     78    4        67        40      40          7   
1   10/2/22  Sun     78    3        29        23      61         46   
2   10/3/22  Mon     78    3        25        22      26         50   
3   10/4/22  Tue     78    2        33        23      15         43   
4   10/5/22  Wed     78    2        40        29      22         36   

  Group Block Group Picked Up TransNGTD TransGTD  Occupied OccPercent  \
0                                                       67    90.54 %   
1                                                       29    38.67 %   
2                                                       25    34.67 %   
3                                                       33    43.42 %   
4                                                       40    52.63 %   

    RoomRev RevPAR     ADR      Ppl  
0  7,163.51  96.80  106.92  117 / 5  
1  2,669.72  35.60   92.06   39 / 4  
2  2,165.42  28.87  

In [12]:
# Clean up and rename columns (particularly 'IDS_DATE' to 'Date')
hotel_data.columns = hotel_data.columns.str.strip().str.replace('﻿', '')  # Remove any unwanted characters
hotel_data.rename(columns={'IDS_DATE': 'Date'}, inplace=True)

# Select the necessary columns
required_columns = ['Date', 'Arrivals', 'Occupied', 'OccPercent', 'RoomRev', 'RevPAR', 'ADR']
hotel_data = hotel_data[required_columns]

# Display the first few rows after renaming
print(hotel_data.head())


      Date  Arrivals  Occupied OccPercent   RoomRev RevPAR     ADR
0  10/1/22        40        67    90.54 %  7,163.51  96.80  106.92
1  10/2/22        23        29    38.67 %  2,669.72  35.60   92.06
2  10/3/22        22        25    34.67 %  2,165.42  28.87   83.29
3  10/4/22        23        33    43.42 %  2,981.50  39.23   90.35
4  10/5/22        29        40    52.63 %  3,608.22  47.48   90.21


In [13]:
import re

# Convert 'Date' to datetime format
hotel_data['Date'] = pd.to_datetime(hotel_data['Date'], format='%m/%d/%y', errors='coerce')

# Remove percentage signs and convert 'OccPercent' to a numeric type
hotel_data['OccPercent'] = hotel_data['OccPercent'].astype(str).str.replace('%', '').astype(float)

# Function to handle parentheses and commas in 'RoomRev', 'RevPAR', 'ADR'
def convert_to_float(value):
    # Handle negative values in parentheses
    if '(' in value and ')' in value:
        value = '-' + value.replace('(', '').replace(')', '')
    return float(value.replace(',', ''))

# Apply the function to relevant columns
hotel_data['RoomRev'] = hotel_data['RoomRev'].astype(str).apply(convert_to_float)
hotel_data['RevPAR'] = hotel_data['RevPAR'].astype(str).apply(convert_to_float)
hotel_data['ADR'] = hotel_data['ADR'].astype(str).apply(convert_to_float)

# Display cleaned data
print(hotel_data.head())


        Date  Arrivals  Occupied  OccPercent  RoomRev  RevPAR     ADR
0 2022-10-01        40        67       90.54  7163.51   96.80  106.92
1 2022-10-02        23        29       38.67  2669.72   35.60   92.06
2 2022-10-03        22        25       34.67  2165.42   28.87   83.29
3 2022-10-04        23        33       43.42  2981.50   39.23   90.35
4 2022-10-05        29        40       52.63  3608.22   47.48   90.21


In [14]:
# Check for missing values in the data
missing_values = hotel_data.isna().sum()
print(missing_values)


Date          0
Arrivals      0
Occupied      0
OccPercent    0
RoomRev       0
RevPAR        0
ADR           0
dtype: int64


In [15]:
# Save the cleaned data for later use in other scripts
hotel_data.to_csv('cleaned_data.csv', index=False)
print("Cleaned data saved to 'cleaned_data.csv'.")


Cleaned data saved to 'cleaned_data.csv'.
